# Lesson 12: SELECT Statements and Querying Data

This lesson covers essential SQL query operations using a real-world HR (Human Resources) database:
1. **Connecting to an existing database**
2. **Exploring database structure** - Understanding tables, columns, and relationships
3. **Basic SELECT statements** - Retrieving data from single tables
4. **Filtering data** - Using WHERE clauses
5. **Sorting data** - Using ORDER BY
6. **Joining tables** - Combining data from multiple tables
7. **Advanced queries** - Getting employee information across related tables

**Database:** HR Employees Database (departments, employees, jobs, locations, etc.)

**Why This Matters:**
- SELECT is the most common SQL operation you'll use
- Real-world data is almost always spread across multiple tables
- Understanding JOINs is critical for working with relational databases

## Part 1: Setup and Load HR Database

Before we can query data, we need to:
1. **Import necessary Python libraries** for database connections
2. **Create a new database** to hold our HR data
3. **Load the HR schema and data** from a SQL file

### Why Load from SQL File?
- Real-world databases often come as SQL dump files
- Contains complete schema (tables, constraints) and data
- Much faster than creating tables manually
- Ensures everyone has the same starting data

In [ ]:
# Part 1: Complete Database Setup (All-in-One)
# This cell does everything: imports, creates database, loads data, and connects

# Step 1: Import necessary libraries
import psycopg2  # PostgreSQL adapter - allows Python to connect to PostgreSQL databases
import pandas as pd  # Data manipulation library - displays query results as nice tables
from sqlalchemy import create_engine, text  # SQLAlchemy - modern database toolkit (text() for SQL 2.0+)
import subprocess  # Allows us to run shell commands (for loading SQL files)

# Step 2: Database connection parameters
DB_NAME = "hr_database"  # Name of the database we'll create for HR data
DB_USER = "student"  # PostgreSQL username (already set up in your dev container)
DB_PASSWORD = ""  # Password (empty for local dev container)
DB_HOST = "localhost"  # Server location (this machine)
DB_PORT = "5432"  # PostgreSQL default port
SQL_FILE_PATH = "/workspaces/Fall2025-MS3083-Base_Template/databases/hr_employees_fixed.sql"

print("=" * 80)
print("STEP 1: Importing libraries...")
print("✓ Libraries imported successfully!")

# Step 3: Drop and recreate database
print("\nSTEP 2: Creating database...")
try:
    conn = psycopg2.connect(dbname="postgres", user=DB_USER, password=DB_PASSWORD, 
                           host=DB_HOST, port=DB_PORT)
    conn.autocommit = True
    cursor = conn.cursor()
    
    # Terminate all connections to the database first
    # This allows us to drop it even if there are active sessions
    cursor.execute(f"""
        SELECT pg_terminate_backend(pg_stat_activity.pid)
        FROM pg_stat_activity
        WHERE pg_stat_activity.datname = '{DB_NAME}'
          AND pid <> pg_backend_pid();
    """)
    print(f"✓ Terminated existing connections to database")
    
    # Drop existing database
    cursor.execute(f"DROP DATABASE IF EXISTS {DB_NAME}")
    print(f"✓ Dropped existing database (if any)")
    
    # Create fresh database
    cursor.execute(f"CREATE DATABASE {DB_NAME}")
    print(f"✓ Database '{DB_NAME}' created successfully!")
    
    cursor.close()
    conn.close()
except Exception as e:
    print(f"❌ Error creating database: {e}")

# Step 4: Load data from SQL file
print("\nSTEP 3: Loading HR data from SQL file...")
try:
    command = ["psql", "-U", DB_USER, "-d", DB_NAME, "-f", SQL_FILE_PATH, "-q"]
    result = subprocess.run(command, capture_output=True, text=True)
    
    if result.returncode == 0:
        print("✓ HR database loaded successfully!")
    else:
        print(f"❌ Error loading database:")
        print(result.stderr)
except Exception as e:
    print(f"❌ Error: {e}")

# Step 5: Create SQLAlchemy connection
print("\nSTEP 4: Connecting to database with SQLAlchemy...")
connection_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_string)

try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT version();"))
        version = result.fetchone()[0]
        print("✓ Successfully connected to database!")
        print(f"✓ PostgreSQL version: {version.split(',')[0]}")
except Exception as e:
    print(f"❌ Connection error: {e}")

print("\n" + "=" * 80)
print("DATABASE SETUP COMPLETE! Ready to run queries.")
print("=" * 80)

## Part 2: Exploring the Database Structure

Before querying data, we need to understand what tables exist and how they're structured.

### Why Explore First?
- You need to know table names to query them
- Understanding column names helps you write correct queries
- Knowing data types prevents errors
- Seeing relationships helps you write proper JOINs

### PostgreSQL System Catalogs
PostgreSQL stores metadata (information about the database structure) in special tables called **system catalogs**:
- `information_schema.tables` - List of all tables
- `information_schema.columns` - List of all columns
- `pg_catalog` - PostgreSQL-specific system information

### List All Tables in the Database

Let's see what tables are available in our HR database.

**Query Breakdown:**
- `information_schema.tables` - System view containing all tables
- `table_schema` - Which schema the table belongs to
- `table_type = 'BASE TABLE'` - Only real tables (not views)
- `ORDER BY` - Sort results alphabetically

In [ ]:
# Query to list all tables in the database
list_tables_query = """
SELECT 
    table_schema,     -- Schema name (like a folder for tables)
    table_name        -- Name of the table
FROM information_schema.tables
WHERE table_type = 'BASE TABLE'     -- Only actual tables (not views)
  AND table_schema NOT IN ('pg_catalog', 'information_schema')  -- Exclude system tables
ORDER BY table_schema, table_name;  -- Sort by schema, then table name
"""

# Execute query and display results using pandas
# pd.read_sql() executes the query and returns results as a DataFrame
# We wrap the query in text() for SQLAlchemy 2.0+ compatibility
df_tables = pd.read_sql(text(list_tables_query), engine)

print("Tables in HR Database:")
print("=" * 60)
print(df_tables.to_string(index=False))  # Display without row numbers
print(f"\nTotal tables: {len(df_tables)}")

### Examine Table Structure (Columns and Data Types)

Now let's look at the structure of a specific table. We'll start with the **employees** table since it's the most important one in an HR database.

**What We'll See:**
- Column names (what fields are available)
- Data types (text, numbers, dates, etc.)
- Nullable (whether the field is required)
- Default values (automatic values if none provided)

**Why This Matters:**
- You need to know column names to SELECT them
- Data types tell you what kind of values are stored
- Nullable columns might contain NULL values you need to handle

In [ ]:
# Query to get column information for the employees table
table_structure_query = """
SELECT 
    column_name,              -- Name of the column
    data_type,                -- Type of data (varchar, integer, date, etc.)
    character_maximum_length, -- Max length for text fields (NULL for non-text)
    is_nullable,              -- YES if the column can be NULL, NO if required
    column_default            -- Default value (if any)
FROM information_schema.columns
WHERE table_schema = 'hr'       -- Look in the 'hr' schema
  AND table_name = 'employees'  -- Specifically the employees table
ORDER BY ordinal_position;      -- Order by position in table (how columns were defined)
"""

# Execute query and display results
# Wrap in text() for SQLAlchemy 2.0+ compatibility
df_structure = pd.read_sql(text(table_structure_query), engine)

print("Structure of 'employees' table:")
print("=" * 100)
print(df_structure.to_string(index=False))
print(f"\nTotal columns: {len(df_structure)}")

### Quick Peek at Sample Data

Let's look at a few rows from the employees table to see what the actual data looks like.

**The SELECT Statement:**
```sql
SELECT column1, column2, ...  -- Which columns to retrieve
FROM table_name               -- Which table to query
LIMIT n;                      -- How many rows to return
```

**Using `*` (asterisk):**
- `SELECT *` means "select all columns"
- Convenient for exploration
- In production, always specify column names (more efficient)

In [ ]:
# Simple SELECT to see sample employee data
# LIMIT 5 means "only show me the first 5 rows"
sample_data_query = """
SELECT *              -- Select all columns (* = wildcard for "everything")
FROM hr.employees     -- From the employees table in the hr schema
LIMIT 5;              -- Only return first 5 rows
"""

# Execute and display
# Wrap in text() for SQLAlchemy 2.0+ compatibility
df_sample = pd.read_sql(text(sample_data_query), engine)

print("Sample Employee Data (first 5 rows):")
print("=" * 120)
# Transpose the DataFrame to show it vertically (easier to read with many columns)
print(df_sample.T)  # .T transposes (swaps rows and columns)

In [ ]:
# Check how many rows are in each table
verify_query = """
SELECT 
    'employees' as table_name, COUNT(*) as row_count FROM hr.employees
UNION ALL
SELECT 'departments', COUNT(*) FROM hr.departments
UNION ALL
SELECT 'jobs', COUNT(*) FROM hr.jobs
UNION ALL
SELECT 'locations', COUNT(*) FROM hr.locations
UNION ALL
SELECT 'countries', COUNT(*) FROM hr.countries
UNION ALL
SELECT 'regions', COUNT(*) FROM hr.regions
ORDER BY table_name;
"""

# Execute and display row counts
df_verify = pd.read_sql(text(verify_query), engine)

print("Row Counts for HR Tables:")
print("=" * 40)
print(df_verify.to_string(index=False))
print("\n✓ If all counts are 0, the SQL file may not have loaded correctly.")

### Verify Data Was Loaded

Before we look at sample data, let's verify that data was actually loaded into the tables.

## Part 3: Basic SELECT Statements

Now that we understand the database structure, let's learn how to retrieve data.

### The SELECT Statement Anatomy

```sql
SELECT column1, column2, ...    -- What to retrieve
FROM table_name                 -- Where to get it from
WHERE condition                 -- Filter rows (optional)
ORDER BY column                 -- Sort results (optional)
LIMIT n;                        -- Limit results (optional)
```

### Basic Rules:
1. **SELECT and FROM are required** - minimum viable query
2. **WHERE filters rows** - before they're returned
3. **ORDER BY sorts results** - after filtering
4. **LIMIT restricts output** - useful for large datasets
5. **Semicolon ends the statement** - marks the end of the SQL command

### Example 1: Select Specific Columns

Instead of `SELECT *`, we can choose exactly which columns we want.

**Why Select Specific Columns?**
- **Performance**: Less data to transfer
- **Clarity**: Shows exactly what you need
- **Security**: Don't expose sensitive columns unnecessarily

In [ ]:
# Select only specific columns from employees
specific_columns_query = """
SELECT 
    employee_id,      -- Unique identifier for each employee
    first_name,       -- Employee's first name
    last_name,        -- Employee's last name
    email,            -- Employee's email address
    hire_date         -- Date they were hired
FROM hr.employees     -- From the employees table
LIMIT 10;             -- Show first 10 employees
"""

# Execute and display
# Wrap in text() for SQLAlchemy 2.0+ compatibility
df_specific = pd.read_sql(text(specific_columns_query), engine)

print("Employee Names and Contact Info:")
print("=" * 80)
print(df_specific.to_string(index=False))
print(f"\nShowing {len(df_specific)} employees")

### Example 2: Filtering Data with WHERE Clause

The WHERE clause lets you filter which rows are returned.

**Common WHERE Operators:**
- `=` - Equal to
- `>`, `<`, `>=`, `<=` - Comparison operators
- `!=` or `<>` - Not equal to
- `LIKE` - Pattern matching (with % wildcard)
- `IN (...)` - Match any value in a list
- `BETWEEN ... AND ...` - Range of values
- `IS NULL` / `IS NOT NULL` - Check for NULL values
- `AND`, `OR`, `NOT` - Combine conditions

In [ ]:
# First, let's see which departments have employees
check_departments_query = """
SELECT 
    d.department_id,
    d.department_name,
    COUNT(e.employee_id) as employee_count
FROM hr.departments d
LEFT JOIN hr.employees e ON d.department_id = e.department_id
GROUP BY d.department_id, d.department_name
HAVING COUNT(e.employee_id) > 0
ORDER BY employee_count DESC
LIMIT 10;
"""

df_dept_check = pd.read_sql(text(check_departments_query), engine)

print("Departments with Employees:")
print("=" * 60)
print(df_dept_check.to_string(index=False))
print("\n(We'll use one of these department IDs in the next example)")

In [ ]:
# Filter employees by department
# Let's find all employees in a specific department (department 9 - Executive)
filter_query = """
SELECT 
    employee_id,
    first_name,
    last_name,
    department_id,    -- Which department they work in
    salary            -- Their salary
FROM hr.employees
WHERE department_id = 9   -- Only show employees in department 9
ORDER BY last_name;       -- Sort by last name alphabetically
"""

# Execute and display
# Wrap in text() for SQLAlchemy 2.0+ compatibility
df_filtered = pd.read_sql(text(filter_query), engine)

print("Employees in Department 9:")
print("=" * 80)
if len(df_filtered) > 0:
    print(df_filtered.to_string(index=False))
    print(f"\nTotal employees in department 9: {len(df_filtered)}")
else:
    print("No employees found in this department.")

### Example 3: Multiple Filter Conditions

You can combine multiple conditions using AND/OR.

**Logic:**
- `AND` - Both conditions must be true
- `OR` - At least one condition must be true
- Use parentheses `()` to group conditions and control logic

In [ ]:
# Find employees with multiple conditions
# Let's find employees who:
# 1. Make more than $5000
# 2. Were hired after 2000
multiple_conditions_query = """
SELECT 
    employee_id,
    first_name,
    last_name,
    salary,
    hire_date,
    department_id
FROM hr.employees
WHERE salary > 5000              -- Condition 1: Salary greater than 5000
  AND hire_date >= '2000-01-01'  -- Condition 2: Hired on or after Jan 1, 2000
ORDER BY salary DESC;            -- Sort by salary, highest first (DESC = descending)
"""

# Execute and display
# Wrap in text() for SQLAlchemy 2.0+ compatibility
df_multiple = pd.read_sql(text(multiple_conditions_query), engine)

print("High-Earning Employees Hired After 2000:")
print("=" * 90)
print(df_multiple.to_string(index=False))
print(f"\nFound {len(df_multiple)} employees matching criteria")

### Example 4: Pattern Matching with LIKE

The LIKE operator lets you search for patterns in text.

**Wildcards:**
- `%` - Matches any number of characters (including zero)
- `_` - Matches exactly one character

**Examples:**
- `'J%'` - Starts with J
- `'%son'` - Ends with "son"
- `'%man%'` - Contains "man" anywhere
- `'J_n'` - Three letters: starts with J, ends with n

In [ ]:
# Find employees whose last name starts with 'S'
pattern_query = """
SELECT 
    employee_id,
    first_name,
    last_name,
    email
FROM hr.employees
WHERE last_name LIKE 'S%'   -- % is a wildcard: S followed by anything
ORDER BY last_name, first_name;
"""

# Execute and display
# Wrap in text() for SQLAlchemy 2.0+ compatibility
df_pattern = pd.read_sql(text(pattern_query), engine)

print("Employees with Last Names Starting with 'S':")
print("=" * 80)
print(df_pattern.to_string(index=False))
print(f"\nFound {len(df_pattern)} employees")

## Part 4: Joining Tables - Combining Related Data

In relational databases, data is split across multiple tables. To get a complete picture, we need to **JOIN** tables together.

### Why Join Tables?
- **Avoid data duplication** - Store department names once, reference them many times
- **Maintain data integrity** - Update department name in one place
- **Normalize data** - Follow database design best practices

### Types of JOINs:
1. **INNER JOIN** - Only rows that match in both tables
2. **LEFT JOIN** - All rows from left table, matching rows from right (NULL if no match)
3. **RIGHT JOIN** - All rows from right table, matching rows from left
4. **FULL OUTER JOIN** - All rows from both tables

**Most Common:** INNER JOIN and LEFT JOIN

### JOIN Example 1: Employees with Department Names

The `employees` table has a `department_id`, but we want to see the department **name**.

**How JOIN Works:**
```sql
FROM table1
JOIN table2 ON table1.column = table2.column
```

**The ON Clause:**
- Specifies how tables are related
- Usually matches a foreign key to a primary key
- `employees.department_id = departments.department_id`

In [ ]:
# Join employees with departments to show department names
join_departments_query = """
SELECT 
    e.employee_id,           -- e is an alias for employees table
    e.first_name,
    e.last_name,
    e.department_id,         -- The ID number (foreign key)
    d.department_name        -- d is an alias for departments table
FROM hr.employees e          -- Main table with alias 'e'
INNER JOIN hr.departments d  -- Join with departments table (alias 'd')
    ON e.department_id = d.department_id  -- Match on department_id
ORDER BY d.department_name, e.last_name
LIMIT 20;
"""

# Execute and display
# Wrap in text() for SQLAlchemy 2.0+ compatibility
df_join_dept = pd.read_sql(text(join_departments_query), engine)

print("Employees with Their Department Names:")
print("=" * 90)
print(df_join_dept.to_string(index=False))
print(f"\nShowing first 20 of many employees")

### JOIN Example 2: Employees with Job Titles

Now let's add job information. Each employee has a `job_id` that links to the `jobs` table.

**Understanding Aliases:**
- `e` = employees (short and easy to type)
- `j` = jobs
- Required when column names exist in multiple tables
- Makes queries more readable

In [ ]:
# Join employees with jobs to show job titles
join_jobs_query = """
SELECT 
    e.employee_id,
    e.first_name,
    e.last_name,
    e.job_id,              -- The job code (like 'IT_PROG')
    j.job_title,           -- The actual job title (like 'Programmer')
    e.salary,
    j.min_salary,          -- Minimum salary for this job
    j.max_salary           -- Maximum salary for this job
FROM hr.employees e
INNER JOIN hr.jobs j       -- Join with jobs table
    ON e.job_id = j.job_id -- Match on job_id
ORDER BY j.job_title, e.last_name
LIMIT 20;
"""

# Execute and display
# Wrap in text() for SQLAlchemy 2.0+ compatibility
df_join_jobs = pd.read_sql(text(join_jobs_query), engine)

print("Employees with Job Titles and Salary Ranges:")
print("=" * 120)
print(df_join_jobs.to_string(index=False))
print(f"\nShowing first 20 employees")

### JOIN Example 3: Multiple Joins - Complete Employee Information

Let's combine multiple JOINs to get complete employee information:
- Employee details
- Department name
- Job title
- Location information

**Chain of Relationships:**
```
employees → departments → locations → countries
employees → jobs
```

**Reading Multi-JOIN Queries:**
1. Start with the main table (employees)
2. Each JOIN adds one more related table
3. ON clauses define how tables connect
4. Order matters - can't reference a table before it's joined

In [ ]:
# Complex query with multiple JOINs
# Get employee details including department, job, and location
multi_join_query = """
SELECT 
    e.employee_id,
    e.first_name || ' ' || e.last_name AS full_name,  -- Concatenate names
    j.job_title,                    -- From jobs table
    e.salary,
    d.department_name,              -- From departments table
    l.city,                         -- From locations table
    l.state_province,               -- From locations table
    c.country_name                  -- From countries table
FROM hr.employees e
INNER JOIN hr.departments d         -- Join 1: Get department info
    ON e.department_id = d.department_id
INNER JOIN hr.jobs j                -- Join 2: Get job info
    ON e.job_id = j.job_id
INNER JOIN hr.locations l           -- Join 3: Get location info
    ON d.location_id = l.location_id
INNER JOIN hr.countries c           -- Join 4: Get country info
    ON l.country_id = c.country_id
ORDER BY e.employee_id
LIMIT 15;
"""

# Execute and display
# Wrap in text() for SQLAlchemy 2.0+ compatibility
df_multi_join = pd.read_sql(text(multi_join_query), engine)

print("Complete Employee Information (Multiple Joins):")
print("=" * 120)
print(df_multi_join.to_string(index=False))
print(f"\nShowing {len(df_multi_join)} employees with full details")

### Understanding LEFT JOIN vs INNER JOIN

**INNER JOIN:** Only returns rows where there's a match in BOTH tables.
- If an employee has no department, they won't appear

**LEFT JOIN:** Returns ALL rows from the left table, even if there's no match.
- If an employee has no department, they still appear (with NULL for department)

Let's compare them:

In [ ]:
# INNER JOIN - only employees WITH departments
inner_join_query = """
SELECT COUNT(*) as employee_count
FROM hr.employees e
INNER JOIN hr.departments d ON e.department_id = d.department_id;
"""

# LEFT JOIN - ALL employees (even without departments)
left_join_query = """
SELECT COUNT(*) as employee_count
FROM hr.employees e
LEFT JOIN hr.departments d ON e.department_id = d.department_id;
"""

# Execute both queries
# Wrap in text() for SQLAlchemy 2.0+ compatibility
inner_count = pd.read_sql(text(inner_join_query), engine)['employee_count'][0]
left_count = pd.read_sql(text(left_join_query), engine)['employee_count'][0]

print("Comparison of JOIN Types:")
print("=" * 60)
print(f"INNER JOIN result: {inner_count} employees")
print(f"LEFT JOIN result:  {left_count} employees")
print(f"\nDifference: {left_count - inner_count} employees have no department")

## Part 5: Practical Queries - Real-World Scenarios

Let's apply what we've learned to answer common business questions.

### Query 1: Find All Employees in a Specific City

**Business Question:** "Who works in our Seattle office?"

**Tables Needed:**
- employees (employee info)
- departments (links employees to locations)
- locations (has city names)

In [ ]:
# Find all employees in Seattle
city_query = """
SELECT 
    e.first_name,
    e.last_name,
    e.email,
    d.department_name,
    l.city,
    l.state_province
FROM hr.employees e
INNER JOIN hr.departments d ON e.department_id = d.department_id
INNER JOIN hr.locations l ON d.location_id = l.location_id
WHERE l.city = 'Seattle'     -- Filter for Seattle only
ORDER BY d.department_name, e.last_name;
"""

# Wrap in text() for SQLAlchemy 2.0+ compatibility
df_city = pd.read_sql(text(city_query), engine)

print("Employees in Seattle Office:")
print("=" * 100)
if len(df_city) > 0:
    print(df_city.to_string(index=False))
    print(f"\nTotal employees in Seattle: {len(df_city)}")
else:
    print("No employees found in Seattle (try a different city)")

### Query 2: Employees Earning Above Job Average

**Business Question:** "Which employees earn more than the midpoint of their salary range?"

**Logic:**
- Calculate midpoint: `(min_salary + max_salary) / 2`
- Compare employee salary to midpoint
- Use expressions in SELECT and WHERE clauses

In [ ]:
# Find employees earning above their job's midpoint salary
above_average_query = """
SELECT 
    e.first_name,
    e.last_name,
    j.job_title,
    e.salary,
    (j.min_salary + j.max_salary) / 2 AS job_midpoint,  -- Calculate midpoint
    e.salary - (j.min_salary + j.max_salary) / 2 AS above_midpoint  -- How much above
FROM hr.employees e
INNER JOIN hr.jobs j ON e.job_id = j.job_id
WHERE e.salary > (j.min_salary + j.max_salary) / 2  -- Only those above midpoint
ORDER BY above_midpoint DESC  -- Highest above midpoint first
LIMIT 10;
"""

# Wrap in text() for SQLAlchemy 2.0+ compatibility
df_above_avg = pd.read_sql(text(above_average_query), engine)

print("Top 10 Employees Earning Above Their Job Midpoint:")
print("=" * 120)
print(df_above_avg.to_string(index=False))

### Query 3: Department Employee Count and Average Salary

**Business Question:** "How many people work in each department and what's their average salary?"

**New Concepts:**
- `COUNT()` - Aggregate function to count rows
- `AVG()` - Aggregate function to calculate average
- `GROUP BY` - Groups rows by common values
- `HAVING` - Filter groups (like WHERE but for groups)

**Aggregate Functions:**
- `COUNT(*)` - Count all rows
- `AVG(column)` - Average of column values
- `SUM(column)` - Sum of column values
- `MIN(column)` - Minimum value
- `MAX(column)` - Maximum value

In [ ]:
# Count employees and calculate average salary by department
dept_summary_query = """
SELECT 
    d.department_name,
    COUNT(e.employee_id) AS employee_count,      -- Count employees
    AVG(e.salary) AS average_salary,             -- Calculate average salary
    MIN(e.salary) AS min_salary,                 -- Lowest salary in dept
    MAX(e.salary) AS max_salary                  -- Highest salary in dept
FROM hr.departments d
LEFT JOIN hr.employees e ON d.department_id = e.department_id
GROUP BY d.department_id, d.department_name      -- Group by department
HAVING COUNT(e.employee_id) > 0                  -- Only depts with employees
ORDER BY employee_count DESC;                    -- Largest departments first
"""

# Wrap in text() for SQLAlchemy 2.0+ compatibility
df_dept_summary = pd.read_sql(text(dept_summary_query), engine)

print("Department Summary - Employee Count and Salaries:")
print("=" * 120)
print(df_dept_summary.to_string(index=False))
print(f"\nTotal departments with employees: {len(df_dept_summary)}")

## Practice Exercises

Now it's your turn! Try writing queries to answer these questions:

1. **Find all employees hired in a specific year** (e.g., 2005)
2. **List employees with salaries in a specific range** (e.g., between 5000 and 10000)
3. **Find the highest paid employee in each department**
4. **List all job titles and count how many employees have each title**
5. **Find employees who work in countries with 'United' in the name**

Use the cells below to practice!

In [ ]:
# YOUR CODE HERE
# Exercise 1: Find employees hired in a specific year


In [ ]:
# YOUR CODE HERE
# Exercise 2: Employees with salaries in a range


In [ ]:
# YOUR CODE HERE
# Exercise 3: Highest paid employee per department


## Summary

In this lesson, you learned:

### ✓ Database Exploration
- How to list all tables in a database
- How to examine table structure (columns, data types)
- How to peek at sample data

### ✓ SELECT Statements
- Basic SELECT syntax: `SELECT columns FROM table`
- Selecting specific columns vs `SELECT *`
- Using LIMIT to control result size

### ✓ Filtering Data
- WHERE clause for filtering rows
- Comparison operators (=, >, <, etc.)
- Combining conditions with AND/OR
- Pattern matching with LIKE and wildcards (%)

### ✓ Sorting Results
- ORDER BY clause
- Ascending (ASC) vs Descending (DESC)
- Sorting by multiple columns

### ✓ Joining Tables
- Why we need JOINs in relational databases
- INNER JOIN - matching rows only
- LEFT JOIN - all rows from left table
- Multiple JOINs - combining 3+ tables
- Table aliases for cleaner queries

### ✓ Aggregate Functions
- COUNT(), AVG(), MIN(), MAX(), SUM()
- GROUP BY for grouping results
- HAVING for filtering groups

### Key Takeaways:

1. **Always explore before querying** - understand table structure first
2. **Start simple, build complexity** - test queries incrementally
3. **Use aliases** - make queries readable (e = employees, d = departments)
4. **JOINs are essential** - most real queries combine multiple tables
5. **WHERE vs HAVING** - WHERE filters rows, HAVING filters groups
6. **Test with LIMIT** - when exploring large datasets

### Next Steps:

- Practice writing more complex JOINs
- Learn about subqueries and CTEs (Common Table Expressions)
- Explore window functions for advanced analytics
- Study query optimization and indexes